In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h') # - timedelta(hours=1)
current_date = current_date.tz_localize('UTC')
print(f'{current_date=}')

current_date=Timestamp('2024-05-02 15:00:00+0000', tz='UTC')


In [3]:
import src.config as config

In [4]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/684180
Connected. Call `.close()` to terminate connection gracefully.
Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/684180/fs/680003/fv/time_series_hourly_feature_view/version/1
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/684180
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (13.82s) 


AssertionError: Time-series data is not complete. Make sure your feature pipeline is up and runnning.

In [ ]:
features.head(10)

In [ ]:
features.shape

In [ ]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

In [ ]:
predictions['pickup_hour'] = current_date
predictions

In [ ]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [ ]:
feature_group.insert(predictions, write_options={"wait_for_job": False})